In [1]:
from numba import jit
import math

###### First steps

In [6]:
#Here we will use the CPU compilation decorator:

@jit
def hypot(x, y):
    # Implementation from https://en.wikipedia.org/wiki/Hypot
    x = abs(x);
    y = abs(y);
    t = min(x, y);
    x = max(x, y);
    t = t / x;
    return x * math.sqrt(1+t*t)

In [4]:
# The first time we call hypot, the compiler
# is triggered and compiles a machine code implementation for float inputs
hypot(3.0, 4.0)

5.0

In [5]:
# Numba also saves the original Python implementation
# of the function in the .py_func attribute


hypot.py_func(3.0, 4.0)

5.0

###### Benchmaring

Let's first measure the speed of the original Python:

In [9]:
%timeit hypot.py_func(3.0, 4.0)

970 ns ± 17.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [10]:
%timeit hypot(3.0, 4.0)

198 ns ± 2.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Python's built-in is even faster than Numba! 

Why not for this case?

This is because Numba does introduce some overhead to each function call that is larger than the function call overhead of Python itself

###### How does Numba work?

see pic

We can see the result of type inference by using the .inspect_types() method, which prints an annotated version of the source code:

In [16]:
hypot.inspect_types()

hypot (float64, float64)
--------------------------------------------------------------------------------
# File: <ipython-input-6-7b8e1dc134b8>
# --- LINE 3 --- 
# label 0

@jit

# --- LINE 4 --- 

def hypot(x, y):

    # --- LINE 5 --- 

    # Implementation from https://en.wikipedia.org/wiki/Hypot

    # --- LINE 6 --- 
    #   x = arg(0, name=x)  :: float64
    #   y = arg(1, name=y)  :: float64
    #   $0.1 = global(abs: <built-in function abs>)  :: Function(<built-in function abs>)
    #   $0.3 = call $0.1(x, kws=[], args=[Var(x, <ipython-input-6-7b8e1dc134b8> (6))], func=$0.1, vararg=None)  :: (float64,) -> float64
    #   del x
    #   del $0.1
    #   x.1 = $0.3  :: float64
    #   del $0.3

    x = abs(x);

    # --- LINE 7 --- 
    #   $0.4 = global(abs: <built-in function abs>)  :: Function(<built-in function abs>)
    #   $0.6 = call $0.4(y, kws=[], args=[Var(y, <ipython-input-6-7b8e1dc134b8> (6))], func=$0.4, vararg=None)  :: (float64,) -> float64
    #   del y
    #   de

Note that Numba's type names tend to mirror the NumPy type names, so a Python float is a float64 (also called "double precision" in other languages). Taking a look at the data types can sometimes be important in GPU code because the performance of float32 and float64 computations will be very different on CUDA devices. An accidental upcast can dramatically slow down a function.

###### When Things Go Wrong

Numba cannot compile all Python code. Some functions don't have a Numba-translation, and some kinds of Python types can't be efficiently compiled at all (yet). For example, Numba does not support dictionaries (as of this tutorial):

In [17]:
@jit
def cannot_compile(x):
    return x['key']

cannot_compile(dict(key='value'))

'value'

Wait, what happened?? By default, Numba will fall back to a mode, called "object mode," which does not do type-specialization. Object mode exists to enable other Numba functionality, but in many cases, you want Numba to tell you if type inference fails. You can force "nopython mode" (the other compilation mode) by passing arguments to the decorator:

In [19]:
@jit(nopython=True)
def cannot_compile(x):
    return x['key']

cannot_compile(dict(key='value'))

TypingError: Failed at nopython (nopython frontend)
Internal error at <numba.typeinfer.ArgConstraint object at 0x0000026579283D30>:
--%<----------------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\Nuclear\Anaconda3\lib\site-packages\numba\errors.py", line 577, in new_error_context
    yield
  File "C:\Users\Nuclear\Anaconda3\lib\site-packages\numba\typeinfer.py", line 199, in __call__
    assert ty.is_precise()
AssertionError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Nuclear\Anaconda3\lib\site-packages\numba\typeinfer.py", line 142, in propagate
    constraint(typeinfer)
  File "C:\Users\Nuclear\Anaconda3\lib\site-packages\numba\typeinfer.py", line 200, in __call__
    typeinfer.add_type(self.dst, ty, loc=self.loc)
  File "C:\Users\Nuclear\Anaconda3\Lib\contextlib.py", line 77, in __exit__
    self.gen.throw(type, value, traceback)
  File "

Now we get an exception when Numba tries to compile the function, with an error that says:

- argument 0: cannot determine Numba type of <class 'dict'>

###### Exercise 

Below is a function that loops over two input NumPy arrays and puts their sum into the output array. Modify this function to call the hypot function we defined above. We will learn a more efficient way to write such functions in a future section.

(Make sure to execute all the cells in this notebook so that hypot is defined.)

In [35]:
@jit(nopython=True)
def ex1(x, y, out):
    for i in range(x.shape[0]):
        # out[i] = x[i] + y[i] # Wrong!
        out[i] = np.sqrt(x[i]**2 + y[i]**2)

In [36]:
import numpy as np

in1 = np.arange(10, dtype=np.float64)
in2 = 2 * in1 + 1
out = np.empty_like(in1)

print('in1:', in1)
print('in2:', in2)

ex1(in1, in2, out)

print('out:', out)

in1: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
in2: [ 1.  3.  5.  7.  9. 11. 13. 15. 17. 19.]
out: [ 1.          3.16227766  5.38516481  7.61577311  9.8488578  12.08304597
 14.31782106 16.55294536 18.78829423 21.02379604]


In [37]:
# This test will fail until you fix the ex1 function
np.testing.assert_almost_equal(out, np.hypot(in1, in2))